In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import sys
import pprint
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
import numpy as np
import requests
import json

In [2]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [3]:
def make_playlist(song, df):
    scaler = MinMaxScaler()# インスタンスの作成
    scaler.fit(df["loudness"].values.reshape(-1, 1))  
    df["loudness"] = scaler.transform(df["loudness"].values.reshape(-1, 1)) 
    joblib.dump(scaler, 'sscaler.sav') 
    #sscaler =joblib.load('sscaler.sav')
    song["loudness"] = scaler.transform(song["loudness"].reshape(1, -1)).reshape(1,)[0]
    
    df = df[(df["tempo"] > song["tempo"]-5) & (df["tempo"] < song["tempo"] +5) ]
    df = df[df["time_signature"] == song["time_signature"]]
    
    search = df.drop("id",1)
    a=[]
    for k in range(len(song.drop("id"))):
        a.append(song.drop("id")[k])
        qury_search = np.array(a)
    cos = []
    for i in range(len(search)):
        cos.append(np.dot(search.iloc[i,:].values ,qury_search)/(np.linalg.norm(search.iloc[i,:].values) * np.linalg.norm(qury_search)))
    
    df["cos"] = cos
    df = df.sort_values("cos", ascending=False)
    df = df.drop("cos",1)
    
    return df.iloc[0]

In [4]:
client_id = '55e915e6dc2549b29feb68ef4e611e2a'
client_secret = '086669968e7e4892bc08c8d39308afaf'
username="21ulbu67tca2mhvh2jiy3g4da"
#client_credentials_manager = spotipy.oauth2.SpotifyClientCredentials(client_id, client_secret)
#sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [5]:
token = util.prompt_for_user_token(
    username="21ulbu67tca2mhvh2jiy3g4da",
    scope='user-read-playback-state,playlist-read-private,user-modify-playback-state,playlist-modify-public', 
    client_id = '55e915e6dc2549b29feb68ef4e611e2a',
    client_secret = '086669968e7e4892bc08c8d39308afaf',
    redirect_uri="http://localhost:8889/callback"
)
sp = spotipy.Spotify(auth=token)

In [6]:
df = pd.read_csv("spotipy.csv")
df = df.drop(["Unnamed: 0","analysis_url","track_href","uri","duration_ms","type"],1)
df = df.drop_duplicates(keep=False)

In [7]:
#一番初めの曲
qury = df.iloc[10]
df = df.drop(10)

In [14]:
name = "RITMO"

In [29]:
id_list = []
name_list = []
result = sp.search(q='track:'+name, limit=5, offset=0, type='track', market=None)
for track in result['tracks']['items']:
    id_list.append(track['id'])
    name_list.append(track["name"])

featur = []
for k in range(len(id_list)):
    features = sp.audio_features(id_list[k])
    featur.append(features)
    
df2 = pd.DataFrame(featur[1],columns=['acousticness', 'analysis_url', 'danceability',"duration_ms","energy","id","instrumentalness","liveness","loudness","mode","speechiness" ,"tempo","time_signature","track_href","type", "uri","valence"])
for k in range(len(featur)):
    [item for item in featur[k] if item]
    df2 = pd.concat([pd.DataFrame([item for item in featur[k] if item],columns=['acousticness', 'analysis_url', 'danceability',"duration_ms","energy","id","instrumentalness","liveness","loudness","mode","speechiness" ,"tempo","time_signature","track_href","type", "uri","valence"]), df2])

df2 =df2.iloc[1:,]  
df2 = df2.reset_index()
name_df = pd.DataFrame(name_list)
df2["name"] = name_df
print(df2["name"])

0                                  Ritmo perfeito
1                       RITMO (Bad Boys For Life)
2                       RITMO (Bad Boys For Life)
3                                  Ritmo Original
4    RITMO (Bad Boys For Life) - Steve Aoki Remix
Name: name, dtype: object


In [41]:
#上のindexから選曲
index = 1
qury = df2.iloc[index].drop(["index","analysis_url","track_href","uri","duration_ms","type","name"])

In [42]:
tracs = []
for i in range(20):
    tracs.append(qury["id"])
    qury =  make_playlist(qury,df)
    #playlist.append( "spotify:track:" +qury["id"])
    df = df[df.loc[:,"id"] != qury["id"]]

In [43]:
#Playlist_Name
Playlist_Name = "test"

In [44]:
playlist = sp.user_playlist_create(username,Playlist_Name)
playlist_id = playlist['id']
sp.user_playlist_add_tracks(username, playlist_id, tracs)

{'snapshot_id': 'Myw2ZTIzMzNiZGFhMTE1ZTE1OTQ5MjlkMTBjOThlNTIyOWE5MTZiNjkx'}